In [1]:
import polars as pl
from pathlib import Path
from interview import Penner_solution as ps

In [2]:
path = Path.cwd().joinpath("data/test_subset.csv")


In [3]:
# df=pl.read_csv(path)
# df

In [4]:
ch = ps.ChicagoWeather(path)

In [5]:
df = ch._get_essentials_from_source(ch.path, ch.target_col)
df = ch._process_time_columns(df)

In [6]:
df.columns

['Station Name', 'Air Temperature', 'Date', 'time_12h', 'time_24h']

In [7]:
df = df.with_row_index()

In [10]:
ddf = df.filter(pl.col("Date").str.contains("12/30/2016")).filter(pl.col("Station Name").str.contains("Foster"))

In [11]:
ddf.head

<bound method DataFrame.head of shape: (24, 6)
┌───────┬────────────────────────┬─────────────────┬────────────┬─────────────┬──────────┐
│ index ┆ Station Name           ┆ Air Temperature ┆ Date       ┆ time_12h    ┆ time_24h │
│ ---   ┆ ---                    ┆ ---             ┆ ---        ┆ ---         ┆ ---      │
│ u32   ┆ str                    ┆ f64             ┆ str        ┆ str         ┆ i32      │
╞═══════╪════════════════════════╪═════════════════╪════════════╪═════════════╪══════════╡
│ 72    ┆ Foster Weather Station ┆ 3.17            ┆ 12/30/2016 ┆ 11:00:00 PM ┆ 2300     │
│ 75    ┆ Foster Weather Station ┆ 2.0             ┆ 12/30/2016 ┆ 10:00:00 PM ┆ 2200     │
│ 78    ┆ Foster Weather Station ┆ 1.17            ┆ 12/30/2016 ┆ 09:00:00 PM ┆ 2100     │
│ 81    ┆ Foster Weather Station ┆ 0.78            ┆ 12/30/2016 ┆ 08:00:00 PM ┆ 2000     │
│ 84    ┆ Foster Weather Station ┆ 0.06            ┆ 12/30/2016 ┆ 07:00:00 PM ┆ 1900     │
│ …     ┆ …                      ┆ …       

In [29]:
df.group_by("Station Name", "Date").agg(
    pl.min("Air Temperature").alias("Min Temp"),
    pl.max("Air Temperature").alias("Max Temp"),
    pl.col("Air Temperature").filter(pl.col("time_24h") == pl.min("time_24h")).first().alias("First Temp"),
    pl.col("Air Temperature").filter(pl.col("time_24h") == pl.max("time_24h")).first().alias("Last Temp"),
).sort("Date", "Station Name")

Station Name,Date,Min Temp,Max Temp,First Temp,Last Temp
str,str,f64,f64,f64,f64
"""63rd Street Weather Station""","""12/30/2016""",-2.8,3.6,0.3,0.6
"""Foster Weather Station""","""12/30/2016""",-3.5,3.17,-0.39,0.06
"""Oak Street Weather Station""","""12/30/2016""",-2.3,4.2,0.7,0.8
"""63rd Street Weather Station""","""12/31/2016""",-1.3,5.6,4.4,3.5
"""Foster Weather Station""","""12/31/2016""",-1.56,5.17,3.67,3.0
"""Oak Street Weather Station""","""12/31/2016""",-0.3,6.3,4.9,4.1


In [24]:
df.group_by("Station Name", "Date").agg(
    pl.max("Air Temperature")
)

Station Name,Date,Air Temperature
str,str,f64
"""Foster Weather Station""","""12/30/2016""",3.17
"""63rd Street Weather Station""","""12/30/2016""",3.6
"""Oak Street Weather Station""","""12/30/2016""",4.2
"""63rd Street Weather Station""","""12/31/2016""",5.6
"""Oak Street Weather Station""","""12/31/2016""",6.3
"""Foster Weather Station""","""12/31/2016""",5.17


In [ ]:
t_his = df.group_by("Station Name", "Date", "index").agg(pl.max("Air Temperature")).select("index", "Air Temperature")
t_los = df.group_by("Station Name", "Date", "index").agg(pl.min("Air Temperature")).select("index", "Air Temperature")
print(t_his)
print(t_los)

In [ ]:
df.group_by("Station Name", "Date").agg(pl.col("Air Temperature").first(), pl.max("time_24h"))